In [2]:
!pip install imblearn

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from moabb.datasets import BNCI2014_001  # [1]
from moabb.paradigms import MotorImagery  # [1]

# Set a random seed for reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

############################################################
# 1) DATA LOADING AND PRE-PROCESSING
############################################################

def load_moabb_data(subject_id=1, fmin=8.0, fmax=30.0, tmin=1.0, tmax=4.0):
    """
    Load data from BNCI2014_001 using the moabb library. 
    Return the EEG trials (X) and class labels (y) after basic pre-processing.
    We apply a band-pass filter from fmin to fmax and extract the time segment 
    between tmin and tmax relative to the trial onset[1].
    """
    dataset = BNCI2014_001()  # [1]
    paradigm = MotorImagery(n_classes=2, fmin=fmin, fmax=fmax, tmin=tmin, tmax=tmax)
    X_list, labels, meta = paradigm.get_data(dataset=dataset, subjects=[subject_id])
    
    # Convert list of arrays into a single numpy array of shape (trials, time_points, channels)
    # The original shape is (channels, time), so we transpose to get (time, channels).
    X_np = []
    for arr in X_list:
        X_np.append(arr.transpose(1,0))
    X_np = np.stack(X_np, axis=0)
    
    # Encode labels numerically
    le = LabelEncoder()
    y_np = le.fit_transform(labels)
    return X_np, y_np
    
def split_and_augment_data(X, y, test_ratio=0.2):
    """
    Split data into training and test sets, optionally apply SMOTE 
    to handle class imbalance in the training set.
    """
    # Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_ratio, shuffle=True, random_state=SEED, stratify=y
    )
    
    # Reshape for SMOTE - SMOTE requires 2D arrays
    n_samples, t_points, n_ch = X_train.shape
    X_train_2d = X_train.reshape(n_samples, -1)
    
    # Apply SMOTE for data augmentation
    smote = SMOTE(random_state=SEED)
    X_train_bal, y_train_bal = smote.fit_resample(X_train_2d, y_train)
    
    # Reshape back to (samples, time_points, channels)
    X_train_bal = X_train_bal.reshape(X_train_bal.shape[0], t_points, n_ch)
    
    # Convert to PyTorch tensors
    X_train_torch = torch.tensor(X_train_bal, dtype=torch.float32)
    y_train_torch = torch.tensor(y_train_bal, dtype=torch.long)
    X_test_torch  = torch.tensor(X_test, dtype=torch.float32)
    y_test_torch  = torch.tensor(y_test, dtype=torch.long)
    
    return X_train_torch, y_train_torch, X_test_torch, y_test_torch

############################################################
# 2) BUILD A 1D-CNN MODEL IN PYTORCH
############################################################

class OneDCNN(nn.Module):
    """
    A 1D CNN model inspired by the referenced architecture[1].
    Convolves along the time axis while treating channels as a feature dimension.
    """
    def __init__(self, num_channels, num_classes):
        super(OneDCNN, self).__init__()
        
        # (batch, time, channels) => reshape or permute needed in forward
        # Convolution blocks
        self.conv1 = nn.Conv1d(in_channels=num_channels, out_channels=32, kernel_size=8, padding=4)
        self.bn1   = nn.BatchNorm1d(32)
        
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=8, padding=0)
        self.bn2   = nn.BatchNorm1d(32)
        self.drop2 = nn.Dropout2d(p=0.5)  # SpatialDropout alternative: Dropout2d on channels
        
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=6, padding=0)
        self.avgpool3 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv4 = nn.Conv1d(in_channels=32, out_channels=32, kernel_size=6, padding=0)
        self.drop4 = nn.Dropout2d(p=0.5)
        
        # Fully connected layers
        # We'll flatten after conv4. The exact dimension depends on input size.
        # We'll compute it dynamically in forward if needed.
        self.fc1 = nn.Linear(32*something_placeholder(160, 4), 296)  # We'll fix the dimension after we see an example
        self.fc2 = nn.Linear(296, 148)
        self.fc3 = nn.Linear(148, 74)
        self.fc4 = nn.Linear(74, num_classes)

    def forward(self, x):
        # x shape: (batch, time, channels) => permute to (batch, channels, time)
        x = x.permute(0, 2, 1)
        
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.drop2(x)
        
        x = F.relu(self.conv3(x))
        x = self.avgpool3(x)
        
        x = F.relu(self.conv4(x))
        x = self.drop4(x)
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.5, training=self.training)
        
        x = F.relu(self.fc3(x))
        x = F.dropout(x, p=0.5, training=self.training)
        
        x = self.fc4(x)
        return x

def compute_flatten_size(model, num_channels, time_dim=640):
    """
    Utility to compute the flatten dimension after the last convolution, 
    for a given input size. We can forward a dummy batch to do so.
    """
    with torch.no_grad():
        dummy = torch.zeros(1, time_dim, num_channels)
        out = model(dummy)
        return out.shape[1]

class OneDCNNFlexible(nn.Module):
    """
    Similar model, but we dynamically fix the FC layer sizes after we see 
    the flattened dimension from the conv blocks.
    """
    def __init__(self, num_channels, num_classes):
        super(OneDCNNFlexible, self).__init__()
        
        self.conv1 = nn.Conv1d(num_channels, 32, 8, padding=4)
        self.bn1   = nn.BatchNorm1d(32)
        
        self.conv2 = nn.Conv1d(32, 32, 8, padding=0)
        self.bn2   = nn.BatchNorm1d(32)
        self.drop2 = nn.Dropout2d(p=0.5)
        
        self.conv3 = nn.Conv1d(32, 32, 6, padding=0)
        self.avgpool3 = nn.AvgPool1d(kernel_size=2, stride=2)
        
        self.conv4 = nn.Conv1d(32, 32, 6, padding=0)
        self.drop4 = nn.Dropout2d(p=0.5)
        
        # We will define fc layers in a separate method once we know flatten size
        self.flatten_size = None
        self.fc1 = None
        self.fc2 = None
        self.fc3 = None
        self.fc4 = None
        
        self.num_classes = num_classes
    
    def forward_features(self, x):
        # x: (batch, time, channels) => permute to (batch, channels, time)
        x = x.permute(0, 2, 1)
        
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.drop2(x)
        
        x = F.relu(self.conv3(x))
        x = self.avgpool3(x)
        
        x = F.relu(self.conv4(x))
        x = self.drop4(x)
        
        return x
    
    def forward(self, x):
        x = self.forward_features(x)
        x = x.view(x.size(0), -1)
        
        # Initialize FC layers if not done yet
        if self.fc1 is None:
            in_feats = x.shape[1]
            self.fc1 = nn.Linear(in_feats, 296)
            self.fc2 = nn.Linear(296, 148)
            self.fc3 = nn.Linear(148, 74)
            self.fc4 = nn.Linear(74, self.num_classes)
            # Move them to same device
            self.fc1.to(x.device)
            self.fc2.to(x.device)
            self.fc3.to(x.device)
            self.fc4.to(x.device)
        
        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5, training=self.training)
        
        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.5, training=self.training)
        
        x = F.relu(self.fc3(x))
        x = F.dropout(x, p=0.5, training=self.training)
        
        x = self.fc4(x)
        return x

############################################################
# 3) TRAINING UTILITIES (EARLY STOPPING, TRAIN/EVAL LOOP)
############################################################

class EarlyStopping:
    """
    Early stops the training if validation loss doesn't improve 
    after a given patience.
    """
    def __init__(self, patience=5, min_delta=1e-4):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_val_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_val_loss is None:
            self.best_val_loss = val_loss
        elif val_loss > (self.best_val_loss - self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_val_loss = val_loss
            self.counter = 0

def train_model(model, train_loader, val_loader, epochs=50, lr=1e-3, patience=5, device='cpu'):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    early_stopper = EarlyStopping(patience=patience, min_delta=1e-4)
    
    best_model_state = None
    best_val_acc = 0.0
    
    for epoch in range(1, epochs+1):
        model.train()
        train_loss, train_correct, total = 0.0, 0, 0
        
        for Xb, yb in train_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            optimizer.zero_grad()
            out = model(Xb)
            loss = criterion(out, yb)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * Xb.size(0)
            preds = out.argmax(dim=1)
            train_correct += (preds == yb).sum().item()
            total += Xb.size(0)
        
        train_loss /= total
        train_acc = 100.0 * train_correct / total
        
        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for Xv, yv in val_loader:
                Xv, yv = Xv.to(device), yv.to(device)
                out_v = model(Xv)
                loss_v = criterion(out_v, yv)
                
                val_loss += loss_v.item() * Xv.size(0)
                preds_v = out_v.argmax(dim=1)
                val_correct += (preds_v == yv).sum().item()
                val_total += Xv.size(0)
        val_loss /= val_total
        val_acc = 100.0 * val_correct / val_total
        
        # Print epoch info
        print(f"Epoch {epoch}/{epochs} - Train Loss: {train_loss:.4f} | "
              f"Train Acc: {train_acc:.2f}% | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")
        
        # Check if this is the best model so far
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = model.state_dict()
        
        # Early stopping
        early_stopper(val_loss)
        if early_stopper.early_stop:
            print("Early stopping triggered.")
            break
    
    # Load best model weights
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    return model

def evaluate_model(model, test_loader, device='cpu'):
    model.eval()
    correct = 0
    total = 0
    criterion = nn.CrossEntropyLoss()
    test_loss = 0.0
    
    with torch.no_grad():
        for Xb, yb in test_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            out = model(Xb)
            loss = criterion(out, yb)
            test_loss += loss.item() * Xb.size(0)
            preds = out.argmax(dim=1)
            correct += (preds == yb).sum().item()
            total += Xb.size(0)
            
    avg_loss = test_loss / total
    acc = 100.0 * correct / total
    return avg_loss, acc

############################################################
# 4) MAIN EXPERIMENT PIPELINE
############################################################

def run_experiment(subject_id=1, batch_size=16, device='cpu'):
    # Load data
    print(f"Loading data for subject {subject_id}...")
    X, y = load_moabb_data(subject_id=subject_id, fmin=8, fmax=30, tmin=1, tmax=4)
    
    # Split + SMOTE
    X_train, y_train, X_test, y_test = split_and_augment_data(X, y, test_ratio=0.2)
    
    # Further split train => train/val
    val_ratio = 0.2
    n_train = int((1 - val_ratio) * X_train.shape[0])
    n_val   = X_train.shape[0] - n_train
    
    train_ds, val_ds = random_split(
        TensorDataset(X_train, y_train),
        [n_train, n_val],
        generator=torch.Generator().manual_seed(SEED)
    )
    
    test_ds = TensorDataset(X_test, y_test)
    
    # Create DataLoaders
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False)
    test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False)
    
    num_classes = len(np.unique(y))
    num_channels = X.shape[2]
    
    # Build model (use the flexible approach so we don't worry about layer shapes)
    model = OneDCNNFlexible(num_channels=num_channels, num_classes=num_classes).to(device)
    
    # Train
    print("Starting training...")
    model = train_model(model, train_loader, val_loader, epochs=50, lr=1e-3, patience=5, device=device)
    
    # Evaluate
    print("Evaluating on test set...")
    test_loss, test_acc = evaluate_model(model, test_loader, device=device)
    print(f"Subject {subject_id} - Test Loss: {test_loss:.4f}  |  Test Acc: {test_acc:.2f}%")
    return model, (test_loss, test_acc)

def main():
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    subject_id = 1  # Example single-subject
    model, results = run_experiment(subject_id=subject_id, device=device)
    print(f"Final test accuracy for subject {subject_id}: {results[1]:.2f}%")

if __name__ == "__main__":
    main()


Choosing from all possible events


Loading data for subject 1...


/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 3 – 6 s (baseline off), ~6.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 3 – 6 s (baseline off), ~6.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: UserWarning: warnEpochs <Epochs | 48 events (all good), 3 – 6 s (baseline off), ~6.1 MB, data loaded,
 'left_hand': 12
 'right_hand': 12
 'feet': 12
 'tongue': 12>
  warn(f"warnEpochs {epochs}")
/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:279: Use

Starting training...


/home/vishwa/anaconda3/envs/eeg_proj/lib/python3.10/site-packages/torch/nn/functional.py:1545: UserWarning: dropout2d: Received a 3D input to dropout2d and assuming that channel-wise 1D dropout behavior is desired - input is interpreted as shape (N, C, L), where C is the channel dim. This behavior will change in a future release to interpret the input as one without a batch dimension, i.e. shape (C, H, W). To maintain the 1D channel-wise dropout behavior, please switch to using dropout1d instead.
  warnings.warn(


Epoch 1/50 - Train Loss: 1.3876 | Train Acc: 23.91% | Val Loss: 1.3827 | Val Acc: 21.74%
Epoch 2/50 - Train Loss: 1.3895 | Train Acc: 22.55% | Val Loss: 1.3834 | Val Acc: 25.00%
Epoch 3/50 - Train Loss: 1.3920 | Train Acc: 25.00% | Val Loss: 1.3836 | Val Acc: 28.26%
Epoch 4/50 - Train Loss: 1.3883 | Train Acc: 26.36% | Val Loss: 1.3833 | Val Acc: 26.09%
Epoch 5/50 - Train Loss: 1.3879 | Train Acc: 25.82% | Val Loss: 1.3842 | Val Acc: 25.00%
Epoch 6/50 - Train Loss: 1.3904 | Train Acc: 24.73% | Val Loss: 1.3844 | Val Acc: 21.74%
Early stopping triggered.
Evaluating on test set...
Subject 1 - Test Loss: 1.3875  |  Test Acc: 30.17%
Final test accuracy for subject 1: 30.17%
